If you've time, expand the following cell to read some note. Thanks. 

**Note**, I'm doing some experiment on some more complex data set in my local. And I was making some note on **some fancy technique** during my experiment and thought to share few things here for [this data set](https://www.kaggle.com/alxmamaev/flowers-recognition). 

---

I was making this notebook off line and so I imported my desire pre-trained weights. Please, check out the following to data set. It may come to you handy. :)

- [Pretrained Model Weights (Keras)](https://www.kaggle.com/ipythonx/keras-pretrained-imagenet-weights)
- [EfficientNet Keras Noisy-Student Weights B0-B7](https://www.kaggle.com/ipythonx/efficientnet-keras-noisystudent-weights-b0b7)

The [pre-trained model weights](https://www.kaggle.com/ipythonx/keras-pretrained-imagenet-weights) contains almost all the imagenet pre-trained weights. If you love to use it, please check this [starting notebook](https://www.kaggle.com/ipythonx/keras-all-models-weights-loader) too, you'll find it helpful. 


# Version Update

- Version 1: Evaluate **SeResNeXT50 and EfficientNet B3**
Validation scores are:
```
    - SeResNeXT50    : 0.9117305
    - EfficientNetB3 : 0.9047619
```
- Version 2: Ensemble of **SeResNeXT50 and EfficientNet B3** 
Validation score increased
```
    - Ensemble :  0.93147504
``` 

In [ ]:
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps
!pip install /kaggle/input/keras-pretrained-imagenet-weights/image_classifiers-1.0.0-py3-none-any.whl

# Content

It's important to know the content of this notebook, otherwise it may not come helpful to you at the end and waste your time. **If you're new to deep learning and image classification in general, or want to brush up some old experience, you may stay. :)**

- Multi-class Image Classification with `tf.keras` or `keras`
- Usage of simple `flow_from_directory` data generator 
- Train 2 state-of-the-art models and compute 
    - loss curve, confusion matrix, classification report 
    - roc auc score 
- How to Ensemble these models for better performance. 

## Imports

In [ ]:
import os, numpy, random, pandas, seaborn
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                                        EarlyStopping, ReduceLROnPlateau, 
                                        TensorBoard, CSVLogger)

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import classification_report, confusion_matrix

import efficientnet.tfkeras as efn
from classification_models.tfkeras import Classifiers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Set Param

In [ ]:
# root
train_data_dir = '../input/flowers-recognition/flowers/flowers'

# hyper parameter
SEED       = 1234
dim        = 226, 226   # dimensions of our images.
epochs     = 80         # epochs  
batch_size = 16         # batch size

def seed_all(SEED):
    tf.random.set_seed(SEED)
    random.seed(SEED)
    numpy.random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    
seed_all(SEED)

## Data Generator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.20,
    horizontal_flip=True)

# flow_from_directory
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=dim,
    batch_size=batch_size,
    seed=42, 
    shuffle=True,
    class_mode='categorical', subset='training')

# flow_from_directory
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=dim,
    batch_size=batch_size,
    seed=42,
    shuffle=False,
    class_mode='categorical', subset='validation')

## Callbacks

In [ ]:
# defining some callbacks
def Call_Back():
    # model check point
    checkpoint = ModelCheckpoint('flowertrainedwg/seresnext.h5', 
                                 monitor = 'val_loss', 
                                 verbose = 0, save_best_only=True, 
                                 mode = 'min',
                                 save_weights_only = True)
    
    csv_logger = CSVLogger('flowerhistory/seresnext.csv')

    # reduce learning rate plateau
    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                            patience=5, verbose=1, 
                            mode='auto', 
                            epsilon=0.0001)
    return [checkpoint, rlr, csv_logger]

# calling all callbacks 
callbacks = Call_Back()

## Build Model

I will try on `SeResNeXT50` and `EfficientNetB3` model. The following `create_model` method is same for both backbone model.

In [ ]:
# model arch
def create_model(input_dim, base_model):
    inp = L.Input(input_dim)
    
    curr_output   = base_model(inp)
    curr_output_1 = L.GlobalAveragePooling2D()(curr_output)
    curr_output_2 = L.GlobalMaxPooling2D()(curr_output)
    curr_output   = L.concatenate([curr_output_1, curr_output_2])

    curr_output   = L.BatchNormalization()(curr_output)
    curr_output   = L.Dense(512, activation='elu')(curr_output)
    curr_output   = L.Dropout(0.5)(curr_output)
    curr_output   = L.BatchNormalization()(curr_output)
    curr_output   = L.Dense(252, activation='elu')(curr_output)
    curr_output   = L.Dropout(0.5)(curr_output)
    
    out = L.Dense(5, activation='softmax') (curr_output)
    model = Model(inp, out)
    return model

## Model: SeResNeXT50

In [ ]:
class_wg_root = '/kaggle/input/keras-pretrained-imagenet-weights/'

# backbone: 
SeResNeXT50, preprocess_input = Classifiers.get('seresnext50')
SRNXT = SeResNeXT50(input_shape=(*dim, 3), include_top=False, weights=None)
SRNXT.load_weights(class_wg_root + 'seresnext50_imagenet_1000_no_top.h5')

# build model
model = create_model(input_dim=(*dim, 3), base_model = SRNXT)

# compile it.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

# plot the model structure
plot_model(model, to_file='model.png')

**!Training, Inferencing**

In [ ]:
training = False

if training:
    # fit the model  
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator) // batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=validation_generator,
        validation_steps=len(validation_generator) // batch_size)
else:
    model.load_weights('../input/flowertrainedwg/seresnext.h5')
    history = pandas.read_csv('../input/flowerhistory/seresnext.csv')

**Learning Curve**

In [ ]:
plt.style.use("seaborn")

def plot_log(filename, show=True):

    data = filename

    fig = plt.figure(figsize=(8,10))
    fig.subplots_adjust(top=0.95, bottom=0.05, right=0.95)
    fig.add_subplot(211)
    
    for key in data.keys():
        if key.find('loss') >= 0:  # loss
            plt.plot(data['epoch'].values, data[key].values, label=key)
    plt.legend()
    plt.title('Training and Validtion Loss')

    fig.add_subplot(212)
    for key in data.keys():
        if key.find('acc') >= 0:  # acc
            plt.plot(data['epoch'].values, data[key].values, label=key)
    plt.legend()
    plt.title('Training and Validation Accuracy')

    if show:
        plt.show()
        
# plt
plot_log(history)

**Evaluate**

In [ ]:
validation_generator.reset()
scores = model.evaluate_generator(validation_generator, 
                                  verbose = False,
                                  steps=len(validation_generator)) 
print("Accuracy = ", scores[1])

**Confusion Matrix**

In [ ]:
# set plot figure size for better view
plt.figure(figsize=(10,7))

# reset the generator
validation_generator.reset()
y_pred = model.predict_generator(validation_generator, 
                                 steps=len(validation_generator),
                                 verbose = False)
y_pred = numpy.argmax(y_pred, axis=1)

conf_mat = confusion_matrix(validation_generator.classes, y_pred)
seaborn.heatmap(conf_mat, annot=True, fmt="d", cbar = False,  cmap = plt.cm.Blues)

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

**Classification Report**

In [ ]:
target = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

print('Classification Report')
print('Validation Acc: %2.2f%%' %(100*accuracy_score(validation_generator.classes, y_pred)))
print(classification_report(validation_generator.classes, y_pred, target_names = target))

**ROC AUC**

In [ ]:
# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

print('ROC AUC score:', multiclass_roc_auc_score(validation_generator.classes, y_pred))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.show()

## Model: EfficientNet B3

In [ ]:
efn_wg_root   = '/kaggle/input/efficientnet-keras-noisystudent-weights-b0b7/'

# backbone: 
efnet =  efn.EfficientNetB3(include_top=False, input_shape=(*dim, 3), weights=None)
efnet.load_weights(efn_wg_root + 'efficientnet-b3_noisy-student_notop.h5')

# build model
model = create_model(input_dim=(*dim, 3), base_model = efnet)

# compile it.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.summary()

# plot the model structure
plot_model(model, to_file='model.png')

In [ ]:
if training:
    # fit the model  
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator) // batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=validation_generator,
        validation_steps=len(validation_generator) // batch_size)
else:
    model.load_weights('../input/flowertrainedwg/efficient.h5')
    history = pandas.read_csv('../input/flowerhistory/efficient.csv')

In [ ]:
# plt
plot_log(history)

**Evaluate**

In [ ]:
validation_generator.reset()
scores = model.evaluate_generator(validation_generator, 
                                  verbose = False,
                                  steps=len(validation_generator)) 
print("Accuracy = ", scores[1])

**Confusion Matrix**

In [ ]:
# set plot figure size for better view
plt.figure(figsize=(10,7))

# reset the generator
validation_generator.reset()
y_pred = model.predict_generator(validation_generator, 
                                 steps=len(validation_generator),
                                 verbose = False)
y_pred = numpy.argmax(y_pred, axis=1)

conf_mat = confusion_matrix(validation_generator.classes, y_pred)
seaborn.heatmap(conf_mat, annot=True, fmt="d", cbar = False,  cmap = plt.cm.Blues)

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

**Classification Report**

In [ ]:
target = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

print('Classification Report')
print('Validation Acc: %2.2f%%' %(100*accuracy_score(validation_generator.classes, y_pred)))
print(classification_report(validation_generator.classes, y_pred, target_names = target))

**ROC AUC**

In [ ]:
# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

print('ROC AUC score:', multiclass_roc_auc_score(validation_generator.classes, y_pred))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.show()

# Ensemble (EfficientNetB3 + SeResNeXT50)

In [ ]:
def ensemble(models, model_input):
    
    Models_output=[model(model_input) for model in models]
    Avg = L.average(Models_output)
    
    modelEnsemble = Model(inputs=model_input, outputs=Avg, name='ensemble')
    modelEnsemble.summary()
    modelEnsemble.compile(optimizers.Adam(lr=0.0001),
                          loss='categorical_crossentropy', 
                          metrics=['accuracy'])
    return modelEnsemble

In [ ]:
model_EF = create_model(input_dim=(*dim, 3), base_model = efnet)
model_SE = create_model(input_dim=(*dim, 3), base_model = SRNXT)

models = []

# Load weights 
model_EF.load_weights('../input/flowertrainedwg/efficient.h5')
models.append(model_EF)

model_SE.load_weights('../input/flowertrainedwg/seresnext.h5')
models.append(model_SE)

model_input = L.Input(shape=models[0].input_shape[1:])
ensemble_model = ensemble(models, model_input)

# plot the model structure
plot_model(ensemble_model, to_file='model.png')

In [ ]:
validation_generator.reset()
scores = ensemble_model.evaluate_generator(validation_generator, 
                                  verbose = False,
                                  steps=len(validation_generator)) 
print("Accuracy = ", scores[1])

## Ensemble of SeResNeXT50 and EfficientNetB3
```
- Confusion Metrix , 
- Classification Report , and 
- ROC AUC Graph
```

In [ ]:
# set plot figure size for better view
plt.figure(figsize=(10,7))

# reset the generator
validation_generator.reset()
y_pred = ensemble_model.predict_generator(validation_generator, 
                                 steps=len(validation_generator),
                                 verbose = False)
y_pred = numpy.argmax(y_pred, axis=1)

conf_mat = confusion_matrix(validation_generator.classes, y_pred)
seaborn.heatmap(conf_mat, annot=True, fmt="d", cbar = False,  cmap = plt.cm.Blues)

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
target = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

print('Classification Report')
print('Validation Acc: %2.2f%%' %(100*accuracy_score(validation_generator.classes, y_pred)))
print(classification_report(validation_generator.classes, y_pred, target_names = target))

In [ ]:
# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)

print('ROC AUC score:', multiclass_roc_auc_score(validation_generator.classes, y_pred))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.show()

End for Now. 